In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import pandas as pd
import numpy as np

from IPython.display import display, HTML

##### search font familys can be used by matplotlib

In [2]:
from matplotlib import font_manager
font_set = {f.name for f in font_manager.fontManager.ttflist}
for f in font_set:
    if 'TW' in f or 'CN' in f:
        print(f)
    

AR PL UMing CN
AR PL UKai CN


In [3]:
mpl.rcParams['font.sans-serif'] = ['AR PL UKai CN']
mpl.rcParams['font.family'] = 'AR PL UKai CN'

In [4]:
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
course_df = pd.read_csv('./data/courses.csv')
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

## 刪除多餘欄位

In [5]:
course_chapter_items_df_ = course_chapter_items_df.drop(['chapter_id', 'chapter_item_id'], axis=1)
course_chapter_items_df_

,course_id,chapter_no,chapter_name,chapter_item_no,chapter_item_name,chapter_item_type,video_length_in_seconds
0,54d5a117065a7e0e00725ac0,1,本課程不分章節,1,拉曲線,LECTURE,578.0
1,54d5a117065a7e0e00725ac0,1,本課程不分章節,2,分割工具,LECTURE,479.0
2,54d5a117065a7e0e00725ac0,1,本課程不分章節,3,油漆桶上色工具,LECTURE,726.0
3,54d5a117065a7e0e00725ac0,1,本課程不分章節,4,幾何圖形分割 X 色塊組合,ASSIGNMENT,56.0
4,54d5d9952246e60a009ec571,1,本課程不分章節,1,作品範例,LECTURE,165.0
...,...,...,...,...,...,...,...
21285,61b941f2e8990300069e4e9e,10,紅酒禮物,2,紅酒禮物：麵團材料、本種攪拌、基本發酵,LECTURE,746.0
21286,61b941f2e8990300069e4e9e,10,紅酒禮物,3,紅酒禮物：分割、整型、烘烤,LECTURE,560.0
21287,61b941f2e8990300069e4e9e,10,紅酒禮物,4,【試做紅酒禮物】,ASSIGNMENT,NaN
21288,61b941f2e8990300069e4e9e,11,Lisa創業心法,1,味覺の感動品牌定位／商品行銷／商品包裝,LECTURE,276.0


## 自建工具

In [19]:
def check_row_number_with_unique_amount(df, df_name, column):
    nrow = df.shape[0]
    nunique = pd.unique(df.loc[:,column]).shape[0]
    msg = f'dataframe {df_name} row number != unique {column} amount'
    assert nrow == nunique, msg
    print(f'dataframe {df_name} row number == unique {column} amount ({nrow} == {nunique})')

def check_unique_items_are_same_set(df1, df1_name, df1_column, df2, df2_name, df2_column):
    df1_set = set(df1.loc[:,df1_column].to_list())
    df2_set = set(df2.loc[:,df2_column].to_list())
    diff = df1_set.symmetric_difference(df2_set)
    msg = f'unique {df1_name}.{df1_column} & unique {df2_name}.{df2_column} are different'
    assert len(diff) == 0, msg
    print(f'unique {df1_name}.{df1_column} & unique {df2_name}.{df2_column} are the same')

def long_series_fold_view(series, nrow=10, index_start_from=0):
    series = series.to_list()
    sta = 0 
    length = len(series)
    clips = []
    col_names = []
    for end in range(0, length, nrow):
        if end == 0:
            continue
        clips.append(series[sta :end])
        col_names.append(f"{sta+index_start_from}_{end+index_start_from-1}")
        sta = end
    else:
        if end < length:
            clips.append(series[end:])
            clips[-1] += [None]*(nrow-(length-end) )
            col_names.append(f"{end}_{end+nrow}")

    df = pd.DataFrame({f"{col_name}": clip for col_name, clip in zip(col_names, clips)})
    df.index = df.index + index_start_from

    return df

def sub_eda1(df, columns):
    for column in columns:
        view=df[[column]].describe()
        display(view)

        N = df[column].describe()['max']
        N = int(N)
        ax = df.plot.hist(column=[column], bins=100, xticks=range(0,N,100), figsize=(20,5), title=column)
        ax.tick_params(axis='x', rotation=45)

def sub_eda2(df, columns):
    for column in columns:
        print(f'column: {columns}')
        print(f'.describe():')
        view = df[[column]].describe()
        display(view)

        print(f'.value_counts().describe():')
        view = pd.DataFrame({column:df[column].value_counts()}).describe()
        display(view)

        print(f'.value_counts() fold view:')
        view = long_series_fold_view(df[column].value_counts(), nrow=10, index_start_from=1)
        display(view)
        
        print(f'.value_counts().plot()')
        df[column].value_counts().plot(figsize=(20,5))
        plt.show()

def series_bar(series, column):
    series.plot.bar(figsize=(20,5), title=f'{column}')
    plt.show()


## NaN 檢查

In [7]:
course_chapter_items_df_nans = course_chapter_items_df.isnull().sum().sum()
course_df_nans = course_df.isnull().sum().sum()
users_df_nans = users_df.isnull().sum().sum()
subgroups_df_nans = subgroups_df.isnull().sum().sum()

train_group_df_nans = train_group_df.isnull().sum().sum()
test_seen_group_df_nans = test_seen_group_df.isnull().sum().sum()
val_seen_group_df_nans = val_seen_group_df.isnull().sum().sum()
test_unseen_group_df_nans = test_unseen_group_df.isnull().sum().sum()
val_unseen_group_df_nans = val_unseen_group_df.isnull().sum().sum()

train_df_nans = train_df.isnull().sum().sum()
test_seen_df_nans = test_seen_df.isnull().sum().sum()
val_seen_df_nans = val_seen_df.isnull().sum().sum()
test_unseen_df_nans = test_unseen_df.isnull().sum().sum()
val_unseen_df_nans = val_unseen_df.isnull().sum().sum()

pd.DataFrame({'df_name':[    
    "course_chapter_items", 
    "course", 
    "users", 
    "subgroups", 
    "train_group",
    "test_seen_group", 
    "val_seen_group", 
    "test_unseen_group", 
    "val_unseen_group", 
    "train", 
    "test_seen", 
    "val_seen", 
    "test_unseen", 
    "val_unseen"], 'nans':[
    course_chapter_items_df_nans, 
    course_df_nans, 
    users_df_nans, 
    subgroups_df_nans, 
    train_group_df_nans,
    test_seen_group_df_nans, 
    val_seen_group_df_nans, 
    test_unseen_group_df_nans, 
    val_unseen_group_df_nans, 
    train_df_nans, 
    test_seen_df_nans, 
    val_seen_df_nans, 
    test_unseen_df_nans, 
    val_unseen_df_nans]})

,df_name,nans
0,course_chapter_items,2029
1,course,69
2,users,293146
3,subgroups,0
4,train_group,705
5,test_seen_group,0
6,val_seen_group,33
7,test_unseen_group,0
8,val_unseen_group,96
9,train,0


In [8]:
course_chapter_items_df.isnull().sum()

course_id                     0
chapter_no                    0
chapter_id                    0
chapter_name                103
chapter_item_id               0
chapter_item_no               0
chapter_item_name             7
chapter_item_type             0
video_length_in_seconds    1919
dtype: int64

In [9]:
users_df.isnull().sum()

user_id                   0
gender                45195
occupation_titles    101510
interests             47810
recreation_names      98631
dtype: int64

In [10]:

train_group_df.isnull().sum()

user_id       0
subgroup    705
dtype: int64

In [11]:
val_seen_group_df.isnull().sum()

user_id      0
subgroup    33
dtype: int64

In [12]:
val_unseen_group_df.isnull().sum()

user_id      0
subgroup    96
dtype: int64

## 基本資料檢查

In [13]:
course_df_cid = pd.unique(course_df.course_id).shape[0]
course_chapter_items_df_cid = pd.unique(course_chapter_items_df.course_id).shape[0]
msg = f'amount of unique coursd_id unmatch in course_df, course_chapter_items_df ({course_df_cid} != {course_chapter_items_df_cid})'
assert course_df_cid == course_chapter_items_df_cid, msg

AssertionError: amount of unique coursd_id unmatch in course_df, course_chapter_items_df (728 != 727)

In [14]:
subgroups_df_sid = pd.unique(subgroups_df.subgroup_id).shape[0]
users_df_uid = pd.unique(users_df.user_id).shape[0]
print(f'unique subgroup id:{subgroups_df_sid}')
print(f'unique user id: {users_df_uid}')


train_df_uid_set = set(train_df.user_id.to_list())
train_group_df_uid_set = set(train_group_df.user_id.to_list())
identical_set = train_df_uid_set.symmetric_difference(train_group_df_uid_set)
msg = f'unique user_id not the same set in train_df, train_group_df'
assert len(identical_set) == 0, msg
print(f'train_df, train_group_df have the same unique user_id set')

check_unique_items_are_same_set(train_df, 'train_df', 'user_id', train_group_df, 'train_group_df', 'user_id')
check_row_number_with_unique_amount(train_df, 'train_df', 'user_id')
check_row_number_with_unique_amount(train_group_df, 'train_group_df', 'user_id')

unique subgroup id:91
unique user id: 130566
train_df, train_group_df have the same unique user_id set
unique train_df.user_id & unique train_group_df.user_id are the same
dataframe train_df row number == unique user_id amount (59737 == 59737)
dataframe train_group_df row number == unique user_id amount (59737 == 59737)
